# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Mercati cafè - Milano

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187849-d1076436-Reviews-Mercanti_Cafe-Milan_Lombardy.html

Home --> Ristoranti --> Milano

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187849-d1076436-Reviews-Mercanti_Cafe-Milan_Lombardy.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187849-d1076436-Reviews-or20-Mercanti_Cafe-Milan_Lombardy.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187849-d1076436-Reviews-or'
URL_p2 = '0-Mercanti_Cafe-Milan_Lombardy.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 39/39 [00:48<00:00,  1.60s/it]

Executed
total_pages:  40
Nomi:  391
Commenti:  391
Voti:  391
Scritto da:  391


In [5]:
# display(all_reviews)

In [6]:
# print(all_names)

In [7]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [8]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 391 
 Recensioni analizzabili: 275 
 Recensioni scartate: 116 



["due sanbitter con pochissimi snack che sapevano di stantio, un the con bustine dozzinali e tre biscottini. 30 euro. Capisco la location, ma un po' di qualità a fronte di un conto così salato....",
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 "Sono stata oggi pomeriggio in compagnia di un'amica. Mi sono fermata per bere una spremuta Il costo è quello della foto in allegato: €15 per due spremute.\nAssurdo",
 'Un pessimo posto in termini di sensazione di pulizia, qualita dei prodotti. Prezzi non giustificati, maleducati e di scarsa compenenza.',
 'Una misera bruschetta, una coca un caffè 27,50 euro : da denuncia\nAvevo poco tempo a pranzo sono entrata perché c’erano tavolini liberi, mai avrei pensato di spendere una simile cifra. Da evitare',
 'sembra di ripetere le stesse cose, ma conviene confermare che nulla cambia.\nun prosecco ( non meglio identificato) e un crodino in piedi al banco 13 euro.\nscortesia, servizi chiusi in piena ora di pranzo- immagino chi desiderava lav

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['maricettab', 'catefabbri', 'amedeoamaro', 'mariamarzia05', 'Marco6767', 'Federica N', 'Stefano M', 'MatteoSpina', '934ginop', '767manuelam', 'anita s', 'walterconca', 'bilfederico', 'Mauro M', 'Mtb12mtb12', 'Patrick R', 'Pakidone', 'Luciana D', 'PAXetVERITAS', 'Luigi_T1958', 'I1451CLgiuseppes', 'Stefano91', 'Luigi D', 'Petro S', 'fi_decorso', 'LorenzoBianco1', 'Montsetvallees', 'natascia s', 'lucagT3238UB', 'Gabriele N', 'laralucasimo', 'GGGGGGG1969', 'Evo L', 'valentinat40', 'Margherita M', 'CAROLE558', 'MarileMilano', 'Marina M', 'sampey888', 'Angelica P', 'barbaras419', 'Luigi S', 'federicagM481BE', 'Luigi D', 'Roberta P', 'Simone B', 'paola006', 'Marcoca365', 'Valeria B', 'Luca G', 'Robytrev', 'paolo francesco p', 'seadas7', '794danielev', 'cloc71', 'Salvus97', 'Renato219', 'Marinabaioni', 'Luca N', 'Gariello', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Isa588', 'kuvac1', 'Loos64', 'Edisy-Edisy', 'giovanni s', 'k

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 'male', 'unknown', 'male', 'female', 'female', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'u

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'U

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  391
Commenti:  391
Voti:  391
Mobile:  391


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  52


[['Marco',
  'M',
  'Un pessimo posto in termini di sensazione di pulizia, qualita dei prodotti. Prezzi non giustificati, maleducati e di scarsa compenenza.',
  '1',
  'Mobile'],
 ['Federica',
  'F',
  'Una misera bruschetta, una coca un caffè 27,50 euro : da denuncia\nAvevo poco tempo a pranzo sono entrata perché c’erano tavolini liberi, mai avrei pensato di spendere una simile cifra. Da evitare',
  '1',
  'PC'],
 ['Stefano',
  'M',
  'sembra di ripetere le stesse cose, ma conviene confermare che nulla cambia.\nun prosecco ( non meglio identificato) e un crodino in piedi al banco 13 euro.\nscortesia, servizi chiusi in piena ora di pranzo- immagino chi desiderava lavarsi le mani prima di mangiare- EVITATE',
  '1',
  'PC'],
 ['Mauro',
  'M',
  'Da non credere. Una presa in giro per chi arriva a Milano e spera nella professionalità e nella onestà. 27 euro per una insalatina , mezza di acqua e 1 caffè......\nEsperienza da non ripetere e sicuramente da sconsigliare',
  '1',
  'PC'],
 ['Pat

### Creiamo la tabella

In [16]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [17]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_21.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_21.xlsx")

display(partial_dataframe)

,Nomi,Genere,Recensioni,Voti,Scritta da
0,Marco,M,Un pessimo posto in termini di sensazione di p...,1,Mobile
1,Federica,F,"Una misera bruschetta, una coca un caffè 27,50...",1,PC
2,Stefano,M,"sembra di ripetere le stesse cose, ma conviene...",1,PC
3,Mauro,M,Da non credere. Una presa in giro per chi arri...,1,PC
4,Patrick,M,Mezzo litro di acqua e un caffè 9.10€. Mi aves...,1,Mobile
5,Luigi,M,ci siamo fermati in una location bella ..ma si...,1,Mobile
6,Margherita,F,Il 9 dicembre ero a Milano con degli amici ci ...,1,Mobile
7,Luigi,M,E scoppiata una bomba nel conto ....Solo a Ven...,1,Mobile
8,Valeria,F,"Posto molto bello ed invitante, ma il ho mangi...",1,Mobile
9,Luca,M,Sono stato qui dopo una cena... da dimenticare...,4,Mobile
